In [1]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
#todo lastgang einfügen für lasten, entscheiden ob temperaturabhängig oder aufgeteilt 
#kosten definieren, stores, links 
#Gebäudetechnik
roof_area = 5500 #in qm
number_transporter = 78 # Anzahl E-Auto
capacitiy_battery_transporter = 40 # Batteriekapazität in kwh
inhouse_pv = 0.35 # Aufdach PV-Leistung maximal nach Haustechnik
investment_cost = 40000000 # in €

#Verbräuche
building_load_hourly = roof_area*0.03 /8760 # in MW
DHW_load_hourly = roof_area*0.008 /8760 # in MW
Thermal_load_hourly = roof_area*0.05 /8760 # in MW
fleet_load_hourly = number_transporter * capacitiy_battery_transporter / 24 / 1000 # in MW

# Profile
renewable_profile_solar = pd.read_csv("ninja_pv_51.1601_13.csv", usecols=["electricity"]) # Lastprofil PV Inhouse
renewable_profile_solar = renewable_profile_solar["electricity"].astype(float)

network = pypsa.Network()
network.set_snapshots(range(8760))

#Knoten
network.add("Bus", name="electricity", carrier="electricity")
network.add("Bus", name="heat_0", carrier="heat")
network.add("Bus", name="heat_1", carrier="heat")

#Generator
network.add("Generator", name="PV", bus="electricity", carrier="electricity", 
            p_nom=inhouse_pv, 
            p_max_pu=renewable_profile_solar, 
            marginal_cost=5)
network.add("Generator", name="Grid", bus="electricity", carrier="electricity", 
            p_nom_expendable=True, 
            marginal_cost=300)

#Lasten
network.add("Load", name="Electric_Load", bus="electricity", p_set=building_load_hourly)
network.add("Load", name="Fleet_Load", bus="electricity", p_set=fleet_load_hourly) 
network.add("Load", name="DHW_Load", bus="heat_0", p_set=DHW_load_hourly) 
network.add("Load", name="Thermal_Load", bus="heat_1", p_set=Thermal_load_hourly) 

#Link
network.add("Link", name="Heat_Pump_0", bus0="electricity", bus1="heat_0", 
            efficiency=3.5, p_nom=10, marginal_cost=300) #Ökostrom
network.add("Link", name="Heat_Pump_1", bus0="electricity", bus1="heat_1", 
            efficiency=3.5, p_nom=10, marginal_cost=300)

#Wasserspeicher
network.add("Store", name="DHW_Storage", bus="heat_0", 
            carrier="heat", e_nom=20, e_cyclic=True, standing_loss=0.005)
network.add("Store", name="Thermal_Storage", bus="heat_1", 
            carrier="heat", e_nom=10, e_cyclic=True)
